## <font color="gray">7.2</font> Marcas de Geoforma
---

Para visualizar dados geográficos, o Altair fornece o tipo de marca `geoshape`. Para criar um mapa básico, podemos criar uma marca `geoshape` e passar nossos dados TopoJSON, que são então desembrulhados em recursos GeoJSON, um para cada país do mundo:


In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

No exemplo acima, o Altair aplica uma cor azul padrão e usa uma projeção de mapa padrão (`mercator`). Podemos personalizar as cores e as larguras das bordas usando propriedades padrão de marca. Usando o método `project`, também podemos adicionar nossa própria projeção de mapa:


In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

Por padrão, o Altair ajusta automaticamente a projeção para que todos os dados caibam dentro da largura e altura do gráfico. Também podemos especificar parâmetros de projeção, como `scale` (nível de zoom) e `translate` (panorama), para personalizar as configurações da projeção. Aqui, ajustamos os parâmetros `scale` e `translate` para focar na Europa:


In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

alt.Chart(...)

_Observe como a resolução de 110m dos dados se torna aparente nesse nível de escala. Para ver linhas de costa e fronteiras mais detalhadas, precisamos de um arquivo de entrada com geometrias mais detalhadas. Ajuste os parâmetros `scale` e `translate` para focar o mapa em outras regiões!_


Até agora, nosso mapa mostra apenas países. Usando o operador `layer`, podemos combinar múltiplos elementos do mapa. O Altair inclui _geradores de dados_ que podemos usar para criar dados para camadas adicionais do mapa:

- O gerador de esfera (`{'sphere': True}`) fornece uma representação GeoJSON da esfera completa da Terra. Podemos criar uma marca `geoshape` adicional que preenche a forma da Terra como uma camada de fundo.
- O gerador de graticule (`{'graticule': ...}`) cria um recurso GeoJSON representando um _graticule_: uma grade formada por linhas de latitude e longitude. O graticule padrão tem meridianos e paralelos a cada 10° entre ±80° de latitude. Para as regiões polares, há meridianos a cada 90°. Essas configurações podem ser personalizadas usando as propriedades `stepMinor` e `stepMajor`.

Vamos sobrepor as marcas de esfera, graticule e país em uma especificação de mapa reutilizável:


In [ ]:
map = alt.layer(
    # use a esfera da Terra como camada base
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # adicione um graticule para as linhas de referência geográficas
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # e depois os países do mundo
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)


Podemos estender o mapa com a projeção desejada e desenhar o resultado. Aqui aplicamos uma [projeção Natural Earth](https://en.wikipedia.org/wiki/Natural_Earth_projection). A camada _sphere_ fornece o fundo azul claro; a camada _graticule_ fornece as linhas de referência geográficas brancas.


In [ ]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)

<font color="gray">7.4</font> Mapas de Símbolos
---

Agora, vamos combinar um mapa base com os dados visualizados como camadas diferentes. Examinaremos o networking dos voôs comerciais nos Estados Unidos, considerando tanto os aeroportos como as rotas dos voôs. Para fazermos isso, precisaremos de três conjuntos de dados diferentes. Para nosso mapa base, usaremos um arquivo do tipo TopoJSON para os Estados Unidos numa resolução de 10m, contendo dados para estados (`states`) e municípios (`counties`):

In [ ]:
usa = data.us_10m.url
usa

Para os aeroportos, vmos usar a base de dados com os campos das coordenadas de `latitude` e `longitude` de cada aeroporto, bem como o código `iata` do aeroporto — por exemplo, `SEA` para o [Aeroporto Internacional de Seattle-Tacoma](https://pt.wikipedia.org/wiki/Aeroporto_Internacional_de_Seattle-Tacoma).

In [ ]:
airports = data.airports.url
airports

Finalmente, vamos usar uma base de dados para as rotas dos voôs, que contem a origem e destino de cada voô (demarcados pelos campos `origin` e `destination`, repectivamente) com os códigos IATA do aeroporto correspondente:

In [ ]:
flights = data.flights_airport.url
flights

Vamos começar criando uma mapa base usando a projeção `albersUsa`, depois adicionando uma camada que plota marcas de círculo para cada aeroporto:

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

São muitos aeroportos! Obviamente, nem todos eles são os grandes centros de fluxo.

Semelhante ao nosso conjunto de dados de códigos postais, nossos dados dos aeroportos incluem pontos que ficam fora da parte continental dos Estados Unido. Então, vemos novamente pontos no canto superior esquerdo. Podemos querer filtrar esses pontos, mas para isso, primeiro precisaremos saber mais sobre a natureza eles.

*Atualize a projeção do mapa acima para *`albers`* — evitando o comportamento idiossincrático do *`albersUsa`* — para que as localizações reais desses pontos adicionais sejam reveladas!*

Agora, em vez de mostrarmos todos os aeroportos de forma indiferente, vamos identificar os principais centros das rotas considerando o número total de rotas que se originam em cada aeroporto. Usaremos o conjunto de dados `routes` como nossa fonte de dados primária: ele contém uma lista de rotas de vôo que podemos combinar para contar o número de rotas para cada aeroporto de origem (no campo `origin`).

No entanto, o conjunto de dados `routes` não inclui as localizações dos aeroportos! Para conectarmos os dados do `routes` com as localizações, precisamos de um novo tipo de transformação de dados: o `lookup`. A transformação `lookup` pega um valor e um campo em um conjunto de dados primário e o usa como uma chave para procurar informações relacionadas em outra tabela. Neste caso, queremos comparar o código do aeroporto de origem em nosso conjunto de dados `routes` com o campo `iata` do conjunto de dados `airports` e, em seguida, extrair os campos de `latitude` e `longitude` correspondentes.

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

*Qual aeroporto dos Estados Unidos tem o maior número de rotas saindo dele?*

Agora que podemos ver os aeroports, gostaríamos de interagir melhor com eles entendendo a estrutura de network do trafégo aéreo. Podemos adicionar a marca gráfica `rule` para representar rotas saindo dos aeroportos de origem até os aeroportos de destino, o que necessita de duas transformações do `lookup` e recuperar as coordenadas para cada ponto desse segmento de reta. Além disso, podemos usar a seleção `single` para filtrar essas rotas tal que apenas as rotas tendo a origem no aeroporto atualmente selecionado seja mostrado.

*Começando do mapa estático acima, você poderia implementar uma versão interativa? Seja livre para pular o código abaixo para se engajar com o mapa interativo primeiro, e pense sobre como você mesmo faria!*

In [ ]:
# interactive selection for origin airport
# select nearest airport to mouse cursor
origin = alt.selection_single(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# shared data reference for lookup transforms
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])

alt.layer(
    # base map of the United States
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # route lines from selected origin airport to destination airports
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filter to selected origin only
    ).transform_lookup(
        lookup='origin', from_=foreign # origin lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # dest lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # size airports by number of outgoing routes
    # 1. aggregate flights-airport data set
    # 2. lookup location data from airports data set
    # 3. remove Puerto Rico (PR) and Virgin Islands (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_selection(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # place smaller circles on top
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

Passe o mouse sobre o mapa para explorar a rede de voos!

## <font color="gray">7.5</font> Mapa Coroplético
---

Um [mapa coroplético](https://pt.wikipedia.org/wiki/Mapa_coropl%C3%A9tico) usa regiões sombreadas ou texturizadas para visualizar valores de dados. Mapas com símbolos dimensionados costumam ser mais precisos para leitura, pois as pessoas tendem a estimar melhor as diferenças proporcionais entre as áreas de círculos do que entre tonalidades de cores. No entanto, os mapas coropléticos são populares na prática e particularmente úteis quando muitos símbolos tornam-se perceptivamente excessivos.

Por exemplo, embora os Estados Unidos tem apenas 50 estados, há milhares de condados dentro desses estados. Vamos construir um mapa coroplético da taxa de desemprego por condado, no ano da recessão de 2008. Em alguns casos, os arquivos de entrada GeoJSON ou TopoJSON podem incluir dados estatísticos que podemos visualizar diretamente. Neste caso, no entanto, temos dois arquivos: nosso arquivo TopoJSON, que inclui os limites dos condados (`usa`), e um arquivo de texto separado que contém as estatísticas de desemprego.

In [ ]:
unemp = data.unemployment.url
unemp

Para integrar nossas fontes de dados, precisaremos novamente usar a transformação de busca (`lookup`), enriquecendo nossos dados geoespaciais (`geoshape`) baseados em TopoJSON com as taxas de desemprego. Em seguida, podemos criar um mapa que inclua uma codificação de cores (`color`) para o campo de taxa (`rate`) recuperado.

In [ ]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True),
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

*Examine as taxas de desemprego por condado. Valores mais altos em Michigan podem estar relacionados à indústria automotiva. Condados nas [Grandes Planícies](https://pt.wikipedia.org/wiki/Grandes_Plan%C3%ADcies) e nos estados das Montanhas apresentam tanto taxas baixas **quanto** altas. Essa variação é significativa ou possivelmente um [artefato de tamanhos amostrais menores](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar mais, tente alterar o limite superior da escala (por exemplo, para `0.2`) para ajustar o mapeamento de cores.*

Uma preocupação central nos mapas coropléticos é a escolha das cores. Acima, usamos o esquema padrão `yellowgreenblue` do Altair para mapas de calor. Abaixo, comparamos outros esquemas, incluindo um esquema *sequencial de tom único* (`teals`), que varia apenas em luminância; um esquema *sequencial multi-matiz* (`viridis`), que varia tanto em luminância quanto em matiz; e um esquema *divergente* (`blueorange`), que usa um ponto médio branco.

In [ ]:
# Função utilitária para gerar uma especificação de mapa para um esquema de cores fornecido.
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

*Quais esquemas de cores você considera mais eficaz? Por que isso pode acontecer? Modifique os mapas acima para usar outros esquemas disponíveis, conforme descrito na [documentação dos Esquemas de Cores do Vega](https://vega.github.io/vega/docs/schemes/).*

## <font color="gray">7.6</font> Projeções Cartográficas
---

Agora que temos alguma experiência na criação de mapas, vamos analisar mais de perto as projeções cartográficas. Conforme explicado pela [Wikipédia](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_cartogr%C3%A1fica).


> <font color="gray"> *Todas as projeções cartográficas necessariamente distorcem a superfície de alguma forma. Dependendo do propósito do mapa, algumas distorções são aceitáveis e outras não são; portanto, diferentes projeções cartográficas existem para preservar algumas propriedades do corpo esférico em detrimento de outras propriedades.*

Algumas das propriedades que podemos querer considerar incluem:

* *Área*: A projeção distorce o tamanho das regiões?
* *Rumo*: Uma linha reta corresponde a uma direção de viagem constante?
* *Distância*: Linhas de comprimento igual correspondem a distâncias iguais no globo?
* *Forma*: A projeção preserva as relações espaciais (ângulos) entre os pontos?

A seleção de uma projeção apropriada depende do caso de uso do mapa. Por exemplo, se estivermos analisando o uso da terra e a extensão das áreas for relevante, podemos escolher uma projeção que preserve a área. Se quisermos visualizar ondas de choque emanando de um terremoto, podemos centralizar o mapa no epicentro do tremor e preservar as distâncias a partir desse ponto. Já se o objetivo for auxiliar a navegação, a preservação do rumo e da forma pode ser mais importante.

Também podemos caracterizar as projeções com base na *superfície de projeção*. As projeções cilíndricas, por exemplo, projetam os pontos da superfície da esfera em um cilindro circundante; ao "desenrolar" o cilindro obtemos o nosso mapa. Como descreveremos a seguir, podemos alternativamente projetar sobre a superfície de um cone (projeções cônicas) ou diretamente em um plano plano (projeções azimutais).

*Vamos primeiro desenvolver nossa intuição interagindo com uma variedade de projeções! [Abra o notebook online de Projeções Cartográficas do Vega-Lite](https://observablehq.com/@vega/vega-lite-cartographic-projections). Use os controles nessa página para selecionar uma projeção e explorar seus parâmetros, como `scale` (escala ou zoom) e translação x/y (panorâmica). Os controles de rotação ([eixo lateral, longitudinal e vertical](https://pt.wikipedia.org/wiki/Eixos_do_avi%C3%A3o)) determinam a orientação do globo em relação à superfície sobre a qual está sendo projetado.*

### <font color="gray">7.6.1 </font> Um Tour pelos Tipos Específicos de Projeções

As [projeções cilíndricas](https://jjallaire.github.io/visualization-curriculum/altair_cartographic.html) mapeiam a esfera para um cilindro circundante e, em seguida, "desenrolam" o cilindro. Se o eixo principal do cilindro estiver orientado no sentido norte-sul, os meridianos são mapeados como linhas retas. As projeções [pseudo-cilíndricas](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representam um meridiano central como uma linha reta, com outros meridianos "curvando-se" para longe do centro.

In [ ]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='equirectangular'),
    minimap.project(type='mercator').properties(title='mercator'),
    minimap.project(type='transverseMercator').properties(title='transverseMercator'),
    minimap.project(type='naturalEarth1').properties(title='naturalEarth1')
).properties(spacing=10).configure_view(stroke=None)

* [Equirectangular](https://en.wikipedia.org/wiki/Equirectangular_projection) (`equirectangular`): Escala os valores das coordenadas de latitude (`lat`) e longitude diretamente (`lon`).
* [Mercator](https://en.wikipedia.org/wiki/Mercator_projection) (`mercator`): Projeta sobre um cilindro, utilizando a longitude (`lon`) diretamente, mas submetendo a latitude (`lat`) a uma transformação não linear. Linhas retas preservam rumos constantes ([linhas de rhumb](https://en.wikipedia.org/wiki/Rhumb_line)), tornando essa projeção bem adequada para navegação. No entanto, áreas no extremo norte ou sul podem ser grandemente distorcidas.
* [Transversa de Mercator](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (`transverseMercator`): Uma projeção Mercator, mas com o cilindro de contorno rotacionado para um eixo transversal. Enquanto a projeção Mercator padrão tem maior precisão ao longo do equador, a projeção Transversa de Mercator é mais precisa ao longo do meridiano central.
* [Terra Natural](https://en.wikipedia.org/wiki/Natural_Earth_projection) (`naturalEarth1`): Uma projeção pseudo-cilíndrica projetada para mostrar o mundo todo em uma única visualização.

As [projeções cônicas](https://en.wikipedia.org/wiki/Map_projection#Conic) mapeiam a esfera para um cone e, em seguida, "desenrolam" o cone para o plano. As projeções cônicas são configuradas por dois *paralelos padrão*, que determinam onde o cone intersecta o globo.

In [ ]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='conicEqualArea'),
    minimap.project(type='conicEquidistant').properties(title='conicEquidistant'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='conicConformal'),
    minimap.project(type='albers').properties(title='albers'),
    minimap.project(type='albersUsa').properties(title='albersUsa')
).properties(spacing=10).configure_view(stroke=None)

* [Cônica de Área Preservada](https://en.wikipedia.org/wiki/Albers_projection) (`conicEqualArea`): Projeção cônica que preserva a área. A forma e a distância não são preservadas, mas a precisão é razoável dentro dos paralelos padrão.

* [Cônica Equidistante](https://en.wikipedia.org/wiki/Equidistant_conic_projection) (`conicEquidistant`): Projeção cônica que preserva a distância ao longo dos meridianos e paralelos padrão.

* [Cônica Conforme](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) (`conicConformal`): Projeção cônica que preserva a forma (ângulos locais), mas não a área ou a distância.

* [Albers](https://en.wikipedia.org/wiki/Albers_projection) (`albers`): Uma variante da projeção cônica de área preservada, com paralelos padrão otimizados para criar mapas dos Estados Unidos.

* [Albers EUA](https://en.wikipedia.org/wiki/Albers_projection) (`albersUsa`): Uma projeção híbrida para os 50 estados dos Estados Unidos da América. Esta projeção une três projeções Albers com parâmetros diferentes para os EUA continentais, Alasca e Havai.

As [**projeções azimutais**](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) mapeiam a esfera diretamente em um plano.

In [ ]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='azimuthalEqualArea'),
    minimap.project(type='azimuthalEquidistant').properties(title='azimuthalEquidistant'),
    minimap.project(type='orthographic').properties(title='orthographic'),
    minimap.project(type='stereographic').properties(title='stereographic'),
    minimap.project(type='gnomonic').properties(title='gnomonic')
).properties(spacing=10).configure_view(stroke=None)

* [Azimutal de Área Preservada](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) (`azimuthalEqualArea`): Projeta com precisão a área em todas as partes do globo, mas não preserva a forma (ângulos locais).

* [Azimutal Equidistante](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection) (`azimuthalEquidistant`): Preserva a distância proporcional do centro da projeção para todos os outros pontos no globo.

* [Ortográfica](https://en.wikipedia.org/wiki/Orthographic_map_projection) (`orthographic`): Projeta um hemisfério visível para um plano distante. Aproximadamente corresponde a uma visão da Terra do espaço exterior.

* [Estereográfica](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_estereogr%C3%A1fica) (`stereographic`): Preserva a forma, mas não a área ou a distância.

* [Gnomômica](https://en.wikipedia.org/wiki/Gnomonic_projection) (`gnomonic`): Projeta a superfície da esfera diretamente para um plano tangente. [Círculos máximos](https://en.wikipedia.org/wiki/Great_circle) ao redor da Terra são projetados como linhas retas, mostrando o caminho mais curto entre os pontos.